In [8]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

letter_list = list(alphabet)
letter_list

In [10]:
url_template = 'https://www.pro-football-reference.com/players/{}/'

def get_WR(letter):
    url = url_template.format(letter) # This formats the url by letter

    response = requests.get(url)
    time.sleep(8)

    soup = BeautifulSoup(response.text, 'html.parser')

    players = []
    div_players = soup.find('div', class_="section_content", id="div_players")
    
    # Loop through p tags
    for p in div_players.find_all('p'):
        meta = p.get_text()
        position = str(meta.split('(')[1].split(')')[0])
        is_wr = 1 if 'WR' in position else 0
        year_span = meta.split(')')[1].strip()
        start_year = int(year_span.split('-')[0])
        hyper_link = p.find('a')['href']
        player_link = f'https://www.pro-football-reference.com{hyper_link}'
        if is_wr == 1 and start_year >= 2018:
            players.append([meta, position, year_span, player_link])
    df = pd.DataFrame(players, columns = ['meta', 'position', 'year_span', 'player_link'])
    return df 

all_players = pd.concat([get_WR(letter) for letter in letter_list], ignore_index=True)
all_players.to_csv("wide_receiver_links.csv", index = False)

In [12]:
output_folder = 'Wr_Players'

wr = pd.read_csv('wide_receiver_links.csv')

# Scrape the tables off the player links
for index, player_link in enumerate(wr['player_link']):
    response = requests.get(player_link)
    time.sleep(8)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Scrape the tables needed
    regular_season_table = soup.find('table', {'class': 'stats_table sortable row_summable per_match_toggle now_sortable', 'id': 'receiving_and_rushing'})
    # advanced_stats_table = soup.find('table', {'class': 'row_summable sortable stats_table now_sortable', 'id': 'detailed_receiving_and_rushing'})

    reg_df = pd.read_html(str(regular_season_table))
    # adv_df = pd.read_html(str(advanced_stats_table))

    player_name = all_players.at[index, 'meta'].split('(')[0].strip()

    file_path = os.path.join(output_folder, f'{player_name}.csv')

    reg_df.to_csv(file_path, index=False)

/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_28613/3691122283.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  reg_df = pd.read_html(str(regular_season_table))


ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.